In [40]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing
from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import Adam

In [41]:
data_train = pd.read_json('train.json')
data_val = pd.read_json('val.json')
MAX_SEQUENCE_LENGTH = data_train.shape[1]

In [42]:
vectors = np.load("GloVe_codeswitch_5k.npy")
words = np.load('5k_vocab_dict.npy').item()
EMBEDDING_DIM = len(vectors[0])

In [43]:
english = "eng"
spanish = "span"
other = "other"

In [44]:
def createExamplesLabels(data):
    examples = []
    labels = []
    num_reviews, review_length = data.shape

    for r in range(num_reviews):
        review_string = ""
        label_vec = []

        for w in range(review_length):

            currWordStruct = data[w][r]

            if currWordStruct == None:
                break
                
            currWord = currWordStruct[0]

            if currWord in words:
                review_string += (" " + currWord)
            else:
                review_string += (" <UNK>")

            if w < (review_length - 1):
                nextWordStruct = data[w + 1][r]
                if nextWordStruct:

                    nextWord = nextWordStruct[0]
                    nextLang = nextWordStruct[1]

                    if nextLang == 'eng':
                        label_vec.append(english)

                    elif nextLang == 'spa':
                        label_vec.append(spanish)

                    elif nextLang == 'eng&spa' or 'eng+spa' or 'spa+eng':
                        label_vec.append(other)

                else:
                    label_vec.append(other)

        labels.append(label_vec)
        examples.append(review_string)
    
    return examples, labels
    

In [45]:
examples_train, labels_train = createExamplesLabels(data_train)
examples_val, labels_val = createExamplesLabels(data_val)

In [46]:
tokenizer = Tokenizer(num_words=len(vectors), filters="", lower=False)
tokenizer.fit_on_texts(examples_train)
sequences_train = tokenizer.texts_to_sequences(examples_train)
sequences_val = tokenizer.texts_to_sequences(examples_val)

word_index = tokenizer.word_index
train_data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
val_data = pad_sequences(sequences_val, maxlen=MAX_SEQUENCE_LENGTH)

In [47]:
embedding_dict = {}
for k,v in words.items():
    embedding_dict[k] = vectors[v]

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embedding_dict.get(word)
    embedding_matrix[i] = embedding_vector

In [48]:
le = preprocessing.LabelEncoder()
le.fit([other, english, spanish])
label_transform_train = np.zeros((len(labels_train), MAX_SEQUENCE_LENGTH, 3))
for i, vec in enumerate(labels_train):

    curr = to_categorical(pad_sequences([le.transform(vec)], maxlen=MAX_SEQUENCE_LENGTH), num_classes = 3)[0]
    label_transform_train[i,:,:] = curr

label_transform_val = np.zeros((len(labels_val), MAX_SEQUENCE_LENGTH, 3))
for i, vec in enumerate(labels_val):

    curr = to_categorical(pad_sequences([le.transform(vec)], maxlen=MAX_SEQUENCE_LENGTH), num_classes = 3)[0]
    label_transform_val[i,:,:] = curr
    
#keys = list(le.classes_)
#vals = le.transform(keys)
#labels_index = dict(zip(keys,vals))

In [49]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

model = Sequential()
model.add(embedding_layer)
model.add(LSTM(200, return_sequences=True, name="LSTM"))
model.add(TimeDistributed(Dense(3, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#model.train_on_batch(data[4:8,:], label_transform[4:8,:,:])
#model.train_on_batch(data[8:12,:], label_transform[8:12,:,:])
#model.predict_on_batch(data[10:20,:])
results = model.fit(train_data, label_transform_train, epochs=6, validation_data = (val_data, label_transform_val), batch_size=100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 61, 300)           1500300   
_________________________________________________________________
LSTM (LSTM)                  (None, 61, 200)           400800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 61, 3)             603       
Total params: 1,901,703
Trainable params: 401,403
Non-trainable params: 1,500,300
_________________________________________________________________
None
Train on 31967 samples, validate on 1000 samples
Epoch 1/6
31967/31967 [==============================] - 89s 3ms/step - loss: 0.1275 - acc: 0.9720 - val_loss: 0.1270 - val_acc: 0.9492
Epoch 2/6
31967/31967 [==============================] - 88s 3ms/step - loss: 0.0683 - acc: 0.9740 - val_loss: 0.1164 - val_acc: 0.9508
Epoch 3/6
31967/31967 [==============================] - 84s 3ms

In [50]:
print(results.history)

{'val_loss': [0.12704526036977767, 0.11638797223567962, 0.10066358149051666, 0.1044807754456997, 0.10011781677603722, 0.09831734970211983], 'val_acc': [0.9491803169250488, 0.950803279876709, 0.9582131266593933, 0.9543934464454651, 0.9622622966766358, 0.9629508256912231], 'loss': [0.12747760998388658, 0.0682584139622943, 0.05653371496036498, 0.04541250440091707, 0.03997279140552525, 0.03578736492392362], 'acc': [0.9720187910311581, 0.9740167519624618, 0.9770362580386013, 0.9816419272653699, 0.9838609187973748, 0.9855701578742261]}
